In [ ]:
import json
import statistics
import folium
import numpy as np
import pandas as pd
import os
import warnings
from pathlib import Path
from IPython.display import Image, display

from cogo import plotting, data_prep, simulation


warnings.filterwarnings('ignore')

In [ ]:
APP_ROOT = Path(os.path.realpath(os.path.expanduser(os.getcwd()))).parents[0]

cogo_data, cogo_stations = data_prep.load_datasets(APP_ROOT)
hourly_trips = data_prep.prepare_hourly_trips(cogo_data)
station_crosslinks = data_prep.build_station_interlinks(cogo_data)

In [ ]:
orchestrator = simulation.Orchestrator(
    cogo_stations,
    station_crosslinks,
    hourly_trips,
    bike_count=350
)

In [ ]:
orchestrator.run_simulation(12, 480)

In [ ]:
# Really, this should be normalized to control for
# number of days a station was in service, but for my
# purposes, I'm just interested in the geographic
# distribution of stations right now
df_agg = plotting.counts_by_hexagon(df=cogo_data, resolution=9)

In [ ]:
m_hex = plotting.choropleth_map(
    df_agg=df_agg,
    name='Departure Count',
    value_col='departure_count',
    with_legend=True)
m_hex = plotting.choropleth_map(
    df_agg=df_agg,
    name='Arrival Count',
    value_col='arrival_count',
    initial_map=m_hex,
    with_legend=True,
    kind='outlier'
)
folium.map.LayerControl('bottomright', collapsed=False).add_to(m_hex)
m_hex.save(str(APP_ROOT / 'output' / 'choropleth_counts.html'))
m_hex